In [ ]:
import os
import tweepy


In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

consumer_key= config.get('KEY','consumer_key')
consumer_secret= config.get('KEY','consumer_secret')
access_token= config.get('KEY','access_token')
access_token_secret= config.get('KEY','access_token_secret')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.get('KEY','json_loc')

auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
image = vision.Image()

def parseImage(imageURL):
    image.source.image_uri = imageURL
    response = client.text_detection(image=image)
    string = ""
    for text in response.text_annotations:
        string =  text.description
        return string


In [ ]:
def getDay(string) :
    s = ""
    for char in string:
        if char.isdigit():
            s += char

    return int(s)

In [ ]:
import re
def parseString(string,mon):
    val = dict()
    try:
        #Date
        match=(re.search("/ ", string))
        i = int(match.end())
        match=(re.search("\)", string))
        j = int(match.start())
        print(string[i:j])
        val['day'] = getDay(string[i:string.find("2021") - 1])
        val['month'] = mon
        val['year'] = '2021'


        #+ve cases
        i = string.find("Positive Cases", string.find("Positive Cases")+1)
        match = (re.search("Tests Conducted", string))
        j = int(match.start())
        val['positive'] = string[i + len("Positive Cases") + 1 : j - 1]

        #Tests
        match=(re.search("Tests Conducted", string))
        i = int(match.end())
        match=(re.search("Positivity Rate", string))
        j = int(match.start())
        val['tests'] = string[i+1:j-1]

        #Recovered
        match=(re.search("%", string))
        i = int(match.end())
        match=(re.search("Deaths", string))
        j = int(match.start())
        val['recovered'] = string[i+1:j-1]

        #Deaths
        match=(re.search("Deaths", string))
        i = int(match.end())
        match=(re.search("COVID-19 Patient Management", string))
        j = int(match.start())
        val['deaths'] = string[i+1:j-1]

        #Tot Vac
        match=(re.search("Beneficiaries vaccinated in last 24 hours", string))
        i = int(match.end())
        match=(re.search("Beneficiaries vaccinated 1", string))
        j = int(match.start())
        val['vaccinated'] = string[i+1:j-1]

        #1st dose
        match=(re.search("dose in last 24 hours", string))
        i = int(match.end())
        match=(re.search("Beneficiaries vaccinated 2", string))
        j = int(match.start())
        val['first_dose'] = string[i+1:j-1]

        #2nd dose
        pat = "dose in last 24 hours"
        pos = string.find(pat)+1
        p1 = string.find(pat, pos+1)
        i = p1 + len(pat)
        match=(re.search("Cumulative beneficiaries vaccinated so far", string))
        j = int(match.start())
        val['second_dose'] = string[i+1:j-1]

        #Active Cases
        match=(re.search("Active Cases", string))
        i = int(match.end())
        match=(re.search("Total Number of Containment Zones as on date", string))
        j = int(match.start())
        val['active_cases'] = string[i+1:j-1]

        #Containment
        match=(re.search("Total Number of Containment Zones as on date: ", string))
        i = int(match.end())
        match=(re.search("Calls received", string))
        j = int(match.start())
        val['zones'] = string[i:j-1]
    except:
        print("/\/\/\ Error caught \/\/\/")
        return None
    print(val)
    return val


In [ ]:
#Add data to MySQL

import mysql.connector as connector
conn = connector.connect(
  host="localhost",
  user="root",
  passwd=config.get('KEY','pass'),
  database="casesheet",
    auth_plugin = 'mysql_native_password'
)
cursor = conn.cursor()

sql = "INSERT INTO cases VALUES ({},{},{},{},{},{},{},{},{},{},{},{})"

def addData(val):
    try:
        # Executing the SQL command
        cursor.execute(sql.format(val['day'],
                                  val['month'],
                                  val['year'],
                                  val['positive'],
                                  val['tests'],
                                  val['recovered'],
                                  val['deaths'],
                                  val['vaccinated'],
                                  val['first_dose'],
                                  val['second_dose'],
                                  val['active_cases'],
                                  val['zones']
                                  ))
        # Commit your changes in the database
        conn.commit()

    except:
        print("### Error caught ### ")
        print(sql.format(val['day'],val['month'],val['year'],val['positive'],val['tests'],val['recovered'],val['deaths'],val['vaccinated'],val['first_dose'],val['second_dose'],val['active_cases'],val['zones']))
        conn.rollback()


In [ ]:
#April Tweets
tweets = ["Eysd7CgU8AoskzN","Eyx_-WeUUAA70JQ","Ey3G4RFVoAQlH9o","Ey8a9VQUcAYJDae","EzBaCycWUAIymQP",
          "EzGjfefVgAkhXt1","EzMHt6FUYAEDq69","EzQvBoyVcAkwjkP","EzWi3tmVgAIpm09","Ezbr0IzVUAYOqBI",
          "Ezg7_j0VUAE_XZE","Ezl7iJdVoAQGhD4","EzrJS2qVcAI6xlj","EzwYaR_VkAESS39","Ez1K5VzUUAUpj7n",
          "Ez6twybUUAQR5WZ","Ez_29t-VIAErDLe","E0Hum4-VcAoc777","E0KBXEcVEAQ5Fcm","E0POc7uVgAE-d2a"]

image_url = "http://pbs.twimg.com/media/{}.jpg"
for tweet in tweets:
    val = parseString(parseImage(image_url.format(tweet)),4)
    if (val != None) :
        addData(val)

conn.close()

In [ ]:
#May Tweets
search_words = "Delhi Health Bulletin (from:CMODelhi)"
date_since = "2021-05-01"

tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1)

image_url = ""
# Iterate and print tweets
for tweet in tweets:
    media = tweet.entities.get('media', [])
    image_url = media[0]['media_url']
    addData(parseString(parseImage(image_url),5))

conn.close()


In [ ]:
import json
#Edit 7th May data -
lis = list()
data = json.load(open("07-05-2021-centers.json"))

for pin in data:
    array = data[pin]
    for cen in array:
        lis.append(cen)


In [ ]:
with open("{}-centers.json".format("07-05-2021"), "w") as outfile:
    json.dump(lis, outfile)

In [ ]:
import configparser
import psycopg2

config = configparser.ConfigParser()
config.read('dwh.cfg')

# Connect to DB and return the connection
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [ ]:
#Add Prev Vac Data
import json
jsons = ["07-05-2021-centers.json","08-05-2021-centers.json","09-05-2021-centers.json"]

query = "INSERT INTO CENTERS(day,month,year,center_id,name,pincode,district_name,vaccine,min_age_limit,fee) " \
        "VALUES({day},{month},{year},{center_id},'{name}',{pincode},'{district_name}','{vaccine}',{min_age_limit},{fee})"


for dat in jsons:
    data = json.load(open(dat))
    for vaues in data:
        str = query.format(**vaues)
        print(str)
        cur.execute(str)
        conn.commit()

conn.close()

In [ ]:
#Add prev cases data

